# 책 '정석으로 배우는 딥러닝'
- RNN, LSTM tenserflow, keras를 이용하여 시퀀스 값 예측하기(pp.225~252)
- 시계열 데이터 준비(pp.225~227)
- 텐서플로로 구현(pp.227~


## 시계열 데이터 준비

In [12]:
import numpy as np

In [13]:
def sin(x, T=100):
    return np.sin(2.0 * np.pi * x /T)

def toy_problem(T=100, ampl=0.05):
    x = np.arange(0, 2*T+1)
    noise = ampl * np.random.uniform(low=-1.0, high = 1.0, size=len(x))
    return sin(x) +noise

In [14]:
T=100
f = toy_problem(T)

In [15]:
length_of_sequences = 2*T
maxlen = 25

data = []
target = []

for i in range(0, length_of_sequences - maxlen +1):
    data.append(f[i: i + maxlen])
    target.append(f[i + maxlen])

In [16]:
X = np.array(data).reshape(len(data), maxlen, 1)

In [17]:
Y = np.array(target).reshape(len(data), 1)

In [18]:
X = np.zeros((len(data), maxlen, 1), dtype=float)
Y = np.zeros((len(data), 1), dtype=float)

for i, seq in enumerate(data):
    for t, value in enumerate(seq):
        X[i, t, 0] =value
    Y[i,0] = target[i]

## 텐서플로로 구현

In [19]:
import tensorflow as tf

In [27]:
def inference(x, n_batch, maxlen=None, n_hidden=None, n_out=None):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)
        
    def bias_variable(shape):
        initial = tf.zeros(shape, dtype=tf.float32)
        return tf.Variable(initial)
    
    cell = tf.contrib.rnn.BasicRNNCell(n_hidden)
    initial_state = cell.zero_state(n_batch, tf.float32)
    
    state = initial_state
    outputs = [] # 과거의 은닉층의 출력을 저장
    with tf.variable_scope('RNN'):
        for t in range(maxlen):
            if t> 0:
                tf.get_variable_scope().reuse_variables()
            (cell_output, state) = cell(x[:, t, :], state)
            outputs.append(cell_output)
            
    output = outputs[-1]
    
    V = weight_variable([n_hidden, n_out])
    c = bias_variable([n_out])
    y = tf.matmul(output, V) + C # 선형 활성
    
    return y

In [23]:
def loss(y, t):
    mse = tf.reduce_mean(tf.square(y - t))
    return mse

In [24]:
def training(loss):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, 
                                       beta1 = 0.9, beta2 = 0.999)
    
    train_step = optimizer.minimize(loss)
    return train_step

In [30]:
n_in = len(X[0][0]) # 1
n_hidden = 20
n_out = len(Y[0]) #1

x = tf.placeholder(tf.float32, shape=[None, maxlen, n_in])
t = tf.placeholder(tf.float32, shape=[None, n_out])
n_batch = tf.placeholder(tf.int32, shape=[])

y = inference(x, n_batch, maxlen=maxlen, n_hidden=n_hidden, n_out=n_out)
loss = loss(y, t)
train_step = training(loss)

ValueError: Variable RNN/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2005, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3616, in create_op
    op_def=op_def)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 788, in _apply_op_helper
    op_def=op_def)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gen_state_ops.py", line 1608, in variable_v2
    shared_name=shared_name, name=name)


In [ ]:
eppo

In [9]:
N_train = int(len(data)* 0.9)
N_validation = len(data) - N_train

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=N_validation)

In [15]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

In [10]:
# from keras.layers.recurrent import simpleRNN
from keras.layers.recurrent import LSTM

In [17]:
n_in = len(X[0][0]) # 1
n_hidden = 20
n_out = len(Y[0]) #1


model = Sequential()
# model.add(SimpleRNN(n_hidden, 
#                     init = weight_variable, 
#                     input_shape = (maxlen, n_out)))
model.add(LSTM(n_hidden, 
                    init = weight_variable, 
                    input_shape = (maxlen, n_in)))
model.add(Dense(n_out, init=weight_variable))
model.add(Activation('linear'))

NameError: name 'weight_variable' is not defined

In [16]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2 = 0.999)
model.compile(loss='mean_squared_error', optimizer = optimizer)

In [ ]:
epochs=500
batch_size=10

history= model.fit(X_train, Y_train, batch_size = batch_size,
                  epochs=epochs, validation_daa = (X_validation, Y_validation),
                  callbacks=[early_stopping])

In [ ]:
truncate = maxlen
Z = X[:1]

original = [f[i] for i in range(maxlen)]
predicted = [None for i in range(maxlen)]

for i in range(length_of_sequences - maxlen +1):
    z_ = Z[-1:]
    y_ = model.predict(z_)
    sequence_ = np.concatemate(
        (z_.reshape(maxlen, n_in)[1:], y_),
        axis=0).reshape(1, maxlen, n_in)
    Z = np.append(Z, sequence_, axis=0)
    predicted.append(y_.reshape(-1))